# 生データ読み込み

In [144]:

import pandas as pd

df = pd.read_csv('../notebooks/dow.csv')
# df = pd.read_excel('../data/キリンシティ時間帯別売り上げ/23年5月~25年5月時間帯別売上.xlsx')

print(df.head())

   店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画  前年実績(前年同曜)  \
0    132  横浜モアーズ  2023-05-01  ランチ  11  62744  25  14   66000       64333   
1    132  横浜モアーズ  2023-05-01  ランチ  12  86418  27  14   54000       52323   
2    132  横浜モアーズ  2023-05-01  ランチ  13  44817  23  14  111000       95962   
3    132  横浜モアーズ  2023-05-01  ランチ  14  29798  17  13   28000       27513   
4    132  横浜モアーズ  2023-05-01  ランチ  15  36012  13   8   12000       13039   

   前年実績(前年同日)   曜日  
0       39805  Mon  
1       29251  Mon  
2       67462  Mon  
3       68343  Mon  
4       37968  Mon  


仮データ変数dffに現状のデータ状態を格納

以降処理中のデータ状態はdffに格納される

In [146]:
def load_data():
    global df, dff

    # dff が定義済み＆空じゃないならそれを使う
    if 'dff' in globals() and isinstance(dff, pd.DataFrame) and not dff.empty:
        print("既存の dff から読み込みます")
        return dff.copy()

    # まだ dff が空 or 定義なし → df をコピー
    if 'df' in globals() and isinstance(df, pd.DataFrame):
        print("df から読み込みます")
        dff = df.copy()
        return dff.copy()

    # 最後の手段としてファイルから読み込む
    try:
        df = pd.read_csv("your_data.csv", encoding="utf-8")
        dff = df.copy()
        print("ファイルから df を読み込み、dff にコピーしました")
        return dff.copy()
    except Exception as e:
        print("エラー: df も dff も存在せず、ファイル読み込みにも失敗しました。", e)
        return None

dff = load_data()
print(dff.head())


既存の dff から読み込みます
   店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画  前年実績(前年同曜)  \
0    132  横浜モアーズ  2023-05-01  ランチ  11  62744  25  14   66000       64333   
1    132  横浜モアーズ  2023-05-01  ランチ  12  86418  27  14   54000       52323   
2    132  横浜モアーズ  2023-05-01  ランチ  13  44817  23  14  111000       95962   
3    132  横浜モアーズ  2023-05-01  ランチ  14  29798  17  13   28000       27513   
4    132  横浜モアーズ  2023-05-01  ランチ  15  36012  13   8   12000       13039   

   前年実績(前年同日)   曜日  
0       39805  Mon  
1       29251  Mon  
2       67462  Mon  
3       68343  Mon  
4       37968  Mon  


過程状態の使い回し用

In [154]:
dfff = dff
print(dfff.head())

  店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画 前年実績(前年同曜)  \
0   132  横浜モアーズ  2023-05-01    L  11  62744  25  14   66000      64333   
1   132  横浜モアーズ  2023-05-01    L  12  86418  27  14   54000      52323   
2   132  横浜モアーズ  2023-05-01    L  13  44817  23  14  111000      95962   
3   132  横浜モアーズ  2023-05-01    L  14  29798  17  13   28000      27513   
4   132  横浜モアーズ  2023-05-01    L  15  36012  13   8   12000      13039   

  前年実績(前年同日)   曜日  
0      39805  Mon  
1      29251  Mon  
2      67462  Mon  
3      68343  Mon  
4      37968  Mon  


不要な行を削除

1. 検索ワードを入力

2. 検索列を入力(Enterを押すと未指定となりデータ全体から検索する)

3. 検索ワードを検索列に部分文字列として持つ行を削除

4. dffに格納

In [175]:
def remove_rows_by_keyword(search_word=None, search_column=None, data=None):
    global dff

    # ── 入力プロンプト ──
    if search_word is None:
        search_word = input("検索ワードを入力してください: ").strip()
        if not search_word:
            print("検索ワードは必須です。処理を中止します。")
            return None

    if search_column is None:
        col = input("検索列名を入力してください（未指定は Enter）: ").strip()
        search_column = col if col else None

    # ── データ読み込み ──
    if data is None:
        data = load_data()
        if data is None:
            print("データが読み込めませんでした。")
            return None

    original_rows = len(data)

    # ── 削除処理 ──
    if search_column is None:
        mask = data.astype(str).apply(lambda x: x.str.contains(search_word, na=False)).any(axis=1)
        result = data[~mask]
        print(f"全列対象で検索ワード '{search_word}' を含む行を削除")
    else:
        if search_column not in data.columns:
            print(f"エラー: 列 '{search_column}' が存在しません")
            print(f"利用可能な列: {list(data.columns)}")
            return data
        mask = data[search_column].astype(str).str.contains(search_word, na=False)
        result = data[~mask]
        print(f"列 '{search_column}' で検索ワード '{search_word}' を含む行を削除")

    removed_rows = original_rows - len(result)
    print(f"削除された行数: {removed_rows} (元の行数: {original_rows} -> 新しい行数: {len(result)})")

    # ── 結果の後処理 ──
    result = result.reset_index(drop=True)
    dff = result.copy()
    print(f"結果を dff 変数に保存しました。形状: {dff.shape}")

    return dff

dff = remove_rows_by_keyword()
# dff = remove_rows_by_keyword(data=dfff)
print(dff.head())

既存の dff から読み込みます
全列対象で検索ワード 'CIAL桜木町' を含む行を削除
削除された行数: 9179 (元の行数: 17974 -> 新しい行数: 8795)
結果を dff 変数に保存しました。形状: (8795, 12)
  店舗コード 店舗名          日付 時間区分  時間     売上  客数 組数   売上計画 前年実績(前年同曜) 前年実績(前年同日)  \
0   175  町田  2023-05-01    L  11  27140   9  6      0      20919      10929   
1   175  町田  2023-05-01    L  12  16321  10  7  32000      33924      32356   
2   175  町田  2023-05-01    L  13  17648   8  6   2000      55165          0   
3   175  町田  2023-05-01    L  14  17838  10  6   4000      22867      12464   
4   175  町田  2023-05-01    L  15  13592   5  4  14000      11865      22128   

    曜日  
0  Mon  
1  Mon  
2  Mon  
3  Mon  
4  Mon  


不要な列を削除

1. 削除したい列名リストを入力(例: A,B,C...)

2. 指定した列を削除

3. dffに格納

In [169]:
def remove_columns(column_names=None, data=None):
    if data is None:
        data = load_data()
        if data is None:
            return None

    # ユーザー入力の取得と処理
    if column_names is None:
        user_input = input("削除したい列名を入力してください（複数ある場合はカンマ区切り）: ").strip()
        if not user_input:
            print("入力がありません。処理を中止します。")
            return data
        column_names = [col.strip() for col in user_input.split(',')]

    # 文字列の場合はリストに変換
    elif isinstance(column_names, str):
        column_names = [column_names]

    # 存在しない列をチェック
    existing_columns = []
    missing_columns = []

    for col in column_names:
        if col in data.columns:
            existing_columns.append(col)
        else:
            missing_columns.append(col)

    if missing_columns:
        print(f"警告: 以下の列が存在しません: {missing_columns}")
        print(f"利用可能な列: {list(data.columns)}")

    if not existing_columns:
        print("削除可能な列がありません")
        print(dff.head())
        return data

    # 列を削除
    result = data.drop(columns=existing_columns)

    print(f"結果を dff 変数に保存しました。形状: {dff.shape}")

    return result

dff = remove_columns()
# dff = remove_columns(data=dfff)
print(dff.head())

警告: 以下の列が存在しません: ['横浜モアーズ']
利用可能な列: ['店舗コード', '店舗名', '日付', '時間区分', '時間', '売上', '客数', '組数', '売上計画', '前年実績(前年同曜)', '前年実績(前年同日)', '曜日']
削除可能な列がありません
  店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画 前年実績(前年同曜)  \
0   132  横浜モアーズ  2023-05-01    L  11  62744  25  14   66000      64333   
1   132  横浜モアーズ  2023-05-01    L  12  86418  27  14   54000      52323   
2   132  横浜モアーズ  2023-05-01    L  13  44817  23  14  111000      95962   
3   132  横浜モアーズ  2023-05-01    L  14  29798  17  13   28000      27513   
4   132  横浜モアーズ  2023-05-01    L  15  36012  13   8   12000      13039   

  前年実績(前年同日)   曜日  
0      39805  Mon  
1      29251  Mon  
2      67462  Mon  
3      68343  Mon  
4      37968  Mon  
  店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画 前年実績(前年同曜)  \
0   132  横浜モアーズ  2023-05-01    L  11  62744  25  14   66000      64333   
1   132  横浜モアーズ  2023-05-01    L  12  86418  27  14   54000      52323   
2   132  横浜モアーズ  2023-05-01    L  13  44817  23  14  111000      95

文字列を置換する

1. 検索ワードを入力する(例: A)

2. 置換後文字列を入力する (例: a)

3. 検索列名を入力する(Enterを押すと未指定となりデータ全体から検索する)

4. 検索列中の検索ワードを置換後文字列に置き換える(例: ABCDE -> aBCDE)

5. dffに格納

In [152]:
def replace_text(search_word=None, replace_word=None, row_filter=None, column_filter=None, data=None):
    global dff

    # データ読み込み
    if data is None:
        data = load_data()
        if data is None:
            return None

    # 検索ワードの入力
    if search_word is None:
        search_word = input("検索したい文字列を入力してください: ").strip()
        if not search_word:
            print("検索文字列が入力されていません。処理を中止します。")
            return data

    # 置換後文字列の入力
    if replace_word is None:
        replace_word = input("置換後の文字列を入力してください: ").strip()
        print(f"置換設定: '{search_word}' → '{replace_word}'")

    # 検索対象列の入力
    if column_filter is None:
        col_input = input("検索対象の列名を入力してください（未指定は Enter で全列対象）: ").strip()
        if col_input:
            # カンマ区切りで複数列指定可
            column_filter = [c.strip() for c in col_input.split(',')]
        else:
            column_filter = None  # 全列対象

    # 処理用コピー
    result = data.copy()

    # 対象列の設定
    if column_filter is None:
        target_columns = result.columns.tolist()
        print("全列を対象に置換します")
    else:
        # 存在する列のみを抽出
        target_columns = [col for col in column_filter if col in result.columns]
        missing_columns = [col for col in column_filter if col not in result.columns]
        if missing_columns:
            print(f"警告: 以下の列が存在しません: {missing_columns}")
        if not target_columns:
            print("対象列がありません")
            return result
        print(f"対象列: {target_columns}")

    # 対象行の設定
    if row_filter is None:
        target_rows = result.index.tolist()
        print("全行を対象に置換します")
    else:
        if isinstance(row_filter, str):
            row_filter = [row_filter]
        target_rows = []
        for row in row_filter:
            if isinstance(row, int) or (isinstance(row, str) and row.isdigit()):
                idx = int(row)
                if idx < len(result):
                    target_rows.append(result.index[idx])
                else:
                    print(f"警告: 行番号 {idx} は存在しません（最大: {len(result)-1}）")
            else:
                if row in result.index:
                    target_rows.append(row)
                else:
                    print(f"警告: 行ラベル '{row}' は存在しません")
        if not target_rows:
            print("対象行がありません")
            return result
        print(f"対象行数: {len(target_rows)}")

    # 置換処理
    replacement_count = 0
    for col in target_columns:
        mask = result.index.isin(target_rows)
        original = result.loc[mask, col].astype(str)
        replaced = original.str.replace(search_word, replace_word, regex=False)
        changed = original != replaced
        replacement_count += changed.sum()
        result.loc[mask, col] = replaced

    print(f"置換完了: '{search_word}' -> '{replace_word}'")
    print(f"置換された箇所数: {replacement_count}")

    # 結果の保存
    dff = result.reset_index(drop=True).copy()
    print(f"結果を dff 変数に保存しました。形状: {dff.shape}")

    return dff

dff = replace_text()
# dff = replace_text(data=dfff)
print(dff.head())

既存の dff から読み込みます
置換設定: 'ディナー' → 'D'
全列を対象に置換します
全行を対象に置換します
置換完了: 'ディナー' -> 'D'
置換された箇所数: 17767
結果を dff 変数に保存しました。形状: (44346, 12)
  店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画 前年実績(前年同曜)  \
0   132  横浜モアーズ  2023-05-01    L  11  62744  25  14   66000      64333   
1   132  横浜モアーズ  2023-05-01    L  12  86418  27  14   54000      52323   
2   132  横浜モアーズ  2023-05-01    L  13  44817  23  14  111000      95962   
3   132  横浜モアーズ  2023-05-01    L  14  29798  17  13   28000      27513   
4   132  横浜モアーズ  2023-05-01    L  15  36012  13   8   12000      13039   

  前年実績(前年同日)   曜日  
0      39805  Mon  
1      29251  Mon  
2      67462  Mon  
3      68343  Mon  
4      37968  Mon  


# 時系列データ特有


日付データ形式変換
1. 日付データの列名を入力
2. 日付形式パターンを入力

    (例: 

    2024/06/07 → %Y/%m/%d, 

    2024年06月07日 → %Y年%m月%d日,
                                
    06-07-2024 → %m-%d-%Y",
                          
    07.06.2024 → %d.%m.%Y

    )

3. dffに格納


In [63]:
def convert_date_format(df, column_name, input_format):
    if column_name not in df.columns:
        raise ValueError(f"列 '{column_name}' が存在しません")

    df_converted = df.copy()

    try:
        df_converted[column_name] = pd.to_datetime(
            df[column_name],
            format=input_format
        ).dt.strftime('%Y-%m-%d')

        print(f"✓ 日付変換完了: {column_name}列 ({input_format} → %Y-%m-%d)")
        return df_converted

    except ValueError as e:
        print(f"✗ 変換エラー: {e}")
        print("入力形式パターンを確認してください")
        return df

def process_dataframe(df):
    try:
        print(f"データ行数: {len(df)}, 列数: {len(df.columns)}")
        print(f"利用可能な列: {', '.join(df.columns)}")

        # 日付列名を入力
        column_name = input("\n日付データの列名を入力してください: ").strip()
        if column_name not in df.columns:
            print(f"✗ 列 '{column_name}' が存在しません")
            return df

        input_format = input("\n元の日付形式パターンを入力してください: ").strip()

        return convert_date_format(df, column_name, input_format)

    except Exception as e:
        print(f"✗ エラーが発生しました: {e}")
        return df

dff = process_dataframe(dff)
print(dff.head())

データ行数: 35171, 列数: 11
利用可能な列: 店舗名, 日付, 時間区分, 時間, 売上, 客数, 組数, 売上計画, 前年実績(前年同曜), 前年実績(前年同日), 曜日
  店舗名          日付  時間区分  時間     売上  客数 組数   売上計画 前年実績(前年同曜) 前年実績(前年同日)   曜日
0  町田  2023-05-01  お昼ご飯  11  27140   9  6      0      20919      10929  Mon
1  町田  2023-05-01  お昼ご飯  12  16321  10  7  32000      33924      32356  Mon
2  町田  2023-05-01  お昼ご飯  13  17648   8  6   2000      55165          0  Mon
3  町田  2023-05-01  お昼ご飯  14  17838  10  6   4000      22867      12464  Mon
4  町田  2023-05-01  お昼ご飯  15  13592   5  4  14000      11865      22128  Mon
✓ 日付変換完了: 日付列 (%Y-%m-%d → %Y-%m-%d)


日付データに曜日列を追加するスクリプト
1. 日付データ列名を入力
2. 保存するcsvファイル名を入力

In [65]:
def add_weekday_to_dataframe(df, date_column, weekday_column_name='曜日'):
    if date_column not in df.columns:
        raise ValueError(f"列 '{date_column}' が存在しません")

    df_with_weekday = df.copy()

    try:
        # 日付列をdatetime型に変換
        temp_date_series = pd.to_datetime(df_with_weekday[date_column])

        # 曜日番号を取得 (0=月曜日, 6=日曜日)
        weekday_numbers = temp_date_series.dt.weekday

        # 英語3文字の曜日マッピング
        weekday_map = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}

        # 曜日列を追加
        df_with_weekday[weekday_column_name] = weekday_numbers.map(weekday_map)

        print(f"✓ 曜日列追加完了: {date_column} → {weekday_column_name}")
        print("  表示形式: Mon, Tue, Wed, Thu, Fri, Sat, Sun")

        return df_with_weekday

    except Exception as e:
        print(f"✗ エラー: {e}")
        return df

def add_weekday_interactive(df):
    """
    対話型で曜日列を追加し、結果を返す
    """
    print("=" * 40)
    print("📅 曜日追加ツール")
    print("=" * 40)

    # DataFrame情報表示
    print(f"データ行数: {len(df)}, 列数: {len(df.columns)}")
    print(f"利用可能な列: {', '.join(df.columns)}")

    # 日付列の選択
    date_column = input("\n日付列名を入力してください: ").strip()

    # 列存在チェック
    if date_column not in df.columns:
        print(f"✗ 列 '{date_column}' が存在しません")
        return df

    # 日付形式チェック
    try:
        sample = df[date_column].dropna().iloc[0]
        pd.to_datetime(sample)
    except:
        print(f"⚠ 警告: '{date_column}' 列の最初の値 '{sample}' は日付として認識できません")
        proceed = input("処理を続行しますか？ (y/n): ").strip().lower()
        if proceed != 'y':
            return df

    # 曜日列名の入力
    weekday_name = input("\n曜日列名を入力してください (デフォルト: '曜日'): ").strip()
    if not weekday_name:
        weekday_name = '曜日'

    # 曜日列を追加
    df_with_weekday = add_weekday_to_dataframe(df, date_column, weekday_name)

    # 結果プレビュー
    print("\n結果プレビュー:")
    preview_cols = [date_column, weekday_name]
    print(df_with_weekday[preview_cols].head(5))

    return df_with_weekday

dff = add_weekday_interactive(dff)
print(dff.head())

📅 曜日追加ツール
データ行数: 35171, 列数: 12
利用可能な列: 店舗名, 日付, 時間区分, 時間, 売上, 客数, 組数, 売上計画, 前年実績(前年同曜), 前年実績(前年同日), 曜日, DOW
✓ 曜日列追加完了: 日付 → dow
  表示形式: Mon, Tue, Wed, Thu, Fri, Sat, Sun

結果プレビュー:
           日付  dow
0  2023-05-01  Mon
1  2023-05-01  Mon
2  2023-05-01  Mon
3  2023-05-01  Mon
4  2023-05-01  Mon
  店舗名          日付  時間区分  時間     売上  客数 組数   売上計画 前年実績(前年同曜) 前年実績(前年同日)   曜日  \
0  町田  2023-05-01  お昼ご飯  11  27140   9  6      0      20919      10929  Mon   
1  町田  2023-05-01  お昼ご飯  12  16321  10  7  32000      33924      32356  Mon   
2  町田  2023-05-01  お昼ご飯  13  17648   8  6   2000      55165          0  Mon   
3  町田  2023-05-01  お昼ご飯  14  17838  10  6   4000      22867      12464  Mon   
4  町田  2023-05-01  お昼ご飯  15  13592   5  4  14000      11865      22128  Mon   

   DOW  
0  Mon  
1  Mon  
2  Mon  
3  Mon  
4  Mon  


変数dffをcsvファイルとして保存

1. 保存したいcsvファイル名(.csv除く)を入力

2. 00_pretreatment.ipynbと同一のディレクトリに保存

In [176]:
def save_to_csv(df, filename=None):

    if filename is None:
        filename = input("保存するファイル名を入力してください（.csvを除く）: ").strip()

    # 入力が空の場合のデフォルト値
    if not filename:
        filename = "output_data"
        print(f"⚠ ファイル名が指定されなかったため、デフォルト名 '{filename}.csv' を使用します")

    if not filename.endswith('.csv'):
        filename += '.csv'

    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✓ CSVファイル保存完了: {filename}")
        print(f"  データ行数: {len(df)}")
        print(f"  列数: {len(df.columns)}")

        # ファイルサイズを表示
        import os
        if os.path.exists(filename):
            file_size = os.path.getsize(filename)
            print(f"  ファイルサイズ: {file_size:,} バイト")

        return filename

    except Exception as e:
        print(f"✗ 保存エラー: {e}")
        return None

if __name__ == "__main__":
    saved_file = save_to_csv(dff)

    if saved_file:
        print(f"\n🎉 ファイルが正常に保存されました: {saved_file}")
    else:
        print("\n❌ ファイルの保存に失敗しました")

✓ CSVファイル保存完了: 町田.csv
  データ行数: 8795
  列数: 12
  ファイルサイズ: 498,313 バイト

🎉 ファイルが正常に保存されました: 町田.csv


csv保存後のdff,dfffを空にする

In [178]:
if 'dff' in globals():
    del dff

In [177]:
if 'dff' in globals():
    del dfff

一時確認用


In [153]:
try:
    print(dff.head())
except:
    print(dfff.head())

  店舗コード     店舗名          日付 時間区分  時間     売上  客数  組数    売上計画 前年実績(前年同曜)  \
0   132  横浜モアーズ  2023-05-01    L  11  62744  25  14   66000      64333   
1   132  横浜モアーズ  2023-05-01    L  12  86418  27  14   54000      52323   
2   132  横浜モアーズ  2023-05-01    L  13  44817  23  14  111000      95962   
3   132  横浜モアーズ  2023-05-01    L  14  29798  17  13   28000      27513   
4   132  横浜モアーズ  2023-05-01    L  15  36012  13   8   12000      13039   

  前年実績(前年同日)   曜日  
0      39805  Mon  
1      29251  Mon  
2      67462  Mon  
3      68343  Mon  
4      37968  Mon  
